In [75]:
# importing the required libraries 
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns


## Diff-in-Diff Analysis Of Drug Related Prescriptions in Florida

In [76]:
# loading data for drug prescriptions in Florida and neighbouring states 
prescriptions_reduced =  pd.read_parquet("../20_intermediate_files/prescriptions_fl.parquet")
prescriptions_reduced

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
527,FL,alachua,12001,12,2007.0,95259.627977,1200112,Alachua County,FL,242685.0,Alachua,FL,12001.0,12.0,Alachua,both
528,FL,alachua,12001,12,2008.0,114675.221922,1200112,Alachua County,FL,244888.0,Alachua,FL,12001.0,12.0,Alachua,both
529,FL,alachua,12001,12,2009.0,141281.048185,1200112,Alachua County,FL,246657.0,Alachua,FL,12001.0,12.0,Alachua,both
530,FL,alachua,12001,12,2010.0,150910.785876,1200112,Alachua County,FL,247614.0,Alachua,FL,12001.0,12.0,Alachua,both
531,FL,alachua,12001,12,2011.0,145750.680899,1200112,Alachua County,FL,249834.0,Alachua,FL,12001.0,12.0,Alachua,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011,NC,hyde,37095,37,2011.0,0.000000,3709537,Hyde County,NC,5798.0,Hyde,NC,37095.0,37.0,Hyde,both
5012,NC,hyde,37095,37,2012.0,0.000000,3709537,Hyde County,NC,5689.0,Hyde,NC,37095.0,37.0,Hyde,both
5013,NC,hyde,37095,37,2013.0,0.000000,3709537,Hyde County,NC,5654.0,Hyde,NC,37095.0,37.0,Hyde,both


In [77]:
prescriptions_reduced.columns

Index(['BUYER_STATE', 'BUYER_COUNTY', 'CountyFIPS_x', 'StateFIPS_x', 'Year',
       'MME', 'FIP_unique', 'County', 'State', 'Population', 'county_test',
       'state_abbrev', 'CountyFIPS_y', 'StateFIPS_y', 'CountyName', '_merge'],
      dtype='object')

In [78]:
# # reducing the number of columns in the drug prescriptions dataset by including only the relevant attributes to create a new dataset

# prescriptions_reduced = prescriptions[
#     [
#         "DRUG_CODE",
#         "DRUG_NAME",
#         "QUANTITY",
#         "UNIT",
#         "STRENGTH",
#         "CALC_BASE_WT_IN_GM",
#         "DOSAGE_UNIT",
#         "Product_Name",
#         "Ingredient_Name",
#         "Measure",
#         "MME_Conversion_Factor",
#         "dos_str",
#         "Year",
#         "Month",
#         "StateFIPS",
#         "StateName",
#         "CountyFIPS",
#         "state_abbrev",
#         "FIP_unique",
#         "Population",
#         "county_test",
#     ]
# ]

In [79]:
prescriptions_reduced
prescriptions_reduced.State.unique()

array(['FL', 'MI', 'NC', 'PA'], dtype=object)

In [80]:
# creating a copy of reduced dataset of prescriptions and converting some of the attributes to appropriate data type

prescriptions_reduced_copy = prescriptions_reduced.copy()

prescriptions_reduced_copy["Year"] = prescriptions_reduced_copy["Year"].astype("int64")
# prescriptions_reduced_copy["DRUG_CODE"] = prescriptions_reduced_copy["DRUG_CODE"].astype("int64")
# prescriptions_reduced_copy["Month"] = prescriptions_reduced_copy["Month"].astype("int64")
# prescriptions_reduced_copy["Population"] = prescriptions_reduced_copy["Population"].astype("int64")

In [81]:
prescriptions_reduced_copy

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
527,FL,alachua,12001,12,2007,95259.627977,1200112,Alachua County,FL,242685.0,Alachua,FL,12001.0,12.0,Alachua,both
528,FL,alachua,12001,12,2008,114675.221922,1200112,Alachua County,FL,244888.0,Alachua,FL,12001.0,12.0,Alachua,both
529,FL,alachua,12001,12,2009,141281.048185,1200112,Alachua County,FL,246657.0,Alachua,FL,12001.0,12.0,Alachua,both
530,FL,alachua,12001,12,2010,150910.785876,1200112,Alachua County,FL,247614.0,Alachua,FL,12001.0,12.0,Alachua,both
531,FL,alachua,12001,12,2011,145750.680899,1200112,Alachua County,FL,249834.0,Alachua,FL,12001.0,12.0,Alachua,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011,NC,hyde,37095,37,2011,0.000000,3709537,Hyde County,NC,5798.0,Hyde,NC,37095.0,37.0,Hyde,both
5012,NC,hyde,37095,37,2012,0.000000,3709537,Hyde County,NC,5689.0,Hyde,NC,37095.0,37.0,Hyde,both
5013,NC,hyde,37095,37,2013,0.000000,3709537,Hyde County,NC,5654.0,Hyde,NC,37095.0,37.0,Hyde,both


In [82]:
# creating a dataset that has all the drug prescriptions in the state of Florida 

florida_prescriptions = prescriptions_reduced_copy[prescriptions_reduced_copy["State"]=="FL"]
# washington_prescriptions_copy = florida_prescriptions.copy()
# florida_prescriptions["shipment_per_100k"] = (
#     (
#         florida_prescriptions["dos_str"]
#         * florida_prescriptions["DOSAGE_UNIT"]
#         * florida_prescriptions["MME_Conversion_Factor"]
#     )
#     / (florida_prescriptions["Population"])
#     * 100000
# )

florida_prescriptions_copy = florida_prescriptions.copy()
florida_prescriptions_copy["shipment_per_100k"] = (
        florida_prescriptions_copy["MME"]
    / (florida_prescriptions_copy["Population"])
    * 100000
)

florida_prescriptions_copy

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge,shipment_per_100k
index,,,,,,,,,,,,,,,,,
527,FL,alachua,12001,12,2007,95259.627977,1200112,Alachua County,FL,242685.0,Alachua,FL,12001.0,12.0,Alachua,both,39252.375704
528,FL,alachua,12001,12,2008,114675.221922,1200112,Alachua County,FL,244888.0,Alachua,FL,12001.0,12.0,Alachua,both,46827.619941
529,FL,alachua,12001,12,2009,141281.048185,1200112,Alachua County,FL,246657.0,Alachua,FL,12001.0,12.0,Alachua,both,57278.345308
530,FL,alachua,12001,12,2010,150910.785876,1200112,Alachua County,FL,247614.0,Alachua,FL,12001.0,12.0,Alachua,both,60945.982810
531,FL,alachua,12001,12,2011,145750.680899,1200112,Alachua County,FL,249834.0,Alachua,FL,12001.0,12.0,Alachua,both,58339.009462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,FL,washington,12133,12,2011,13964.081090,1213312,Washington County,FL,24510.0,Washington,FL,12133.0,12.0,Washington,both,56972.995063
1123,FL,washington,12133,12,2012,15658.135788,1213312,Washington County,FL,24741.0,Washington,FL,12133.0,12.0,Washington,both,63288.208997
1124,FL,washington,12133,12,2013,16216.275325,1213312,Washington County,FL,24504.0,Washington,FL,12133.0,12.0,Washington,both,66178.074293


In [83]:
# calculating and displaying total number of drug prescriptions that took place in florida. results are grouped and displayed year and county wise 

florida_prescriptions_result = florida_prescriptions_copy.groupby(["Year", "county_test"])["shipment_per_100k"].sum().reset_index()
# florida_prescriptions_result = florida_prescriptions.groupby(["Year"])["shipment_per_100k"].sum().reset_index()

florida_prescriptions_result.head()
# def scale_years(year):
#     if year == 2007:
#         return -3
#     if year == 2008:
#         return -2
#     if year == 2009:
#         return -1
#     if year == 2010:
#         return 0
#     if year == 2011:
#         return 1
#     if year == 2012:
#         return 2

# florida_prescriptions_result["year relative to policy"] = florida_prescriptions_result["Year"].apply(lambda x: scale_years(x))
# comp_result["year relative to policy"] = comp_result["Year"].apply(lambda x: scale_years(x))

,Year,county_test,shipment_per_100k
0,2007,Alachua,39252.375704
1,2007,Baker,52623.943931
2,2007,Bay,37047.299888
3,2007,Bradford,55003.869818
4,2007,Brevard,45672.967588


## Diff-in-Diff Analysis Of Opioid shipments in Washington and Other States 

In [84]:
# creating a new dataset, that contains drug related deaths in other neighboring states of Florida - Alabama, Georgia, Mississippi, South Carolina and Tennessee

other_states_shipment = prescriptions_reduced_copy[(prescriptions_reduced_copy["State"] != "FL")]
# other_states_shipment = prescriptions_reduced_copy[(prescriptions_reduced_copy["StateName"] == "Alabama")|(prescriptions_reduced_copy["StateName"] == "Georgia")|(prescriptions_reduced_copy["StateName"] == "Mississippi")|(prescriptions_reduced_copy["StateName"] == "South Carolina")|(prescriptions_reduced_copy["StateName"] == "Tennessee")]

other_states_shipment


,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
1343,MI,alcona,26001,26,2007,1625.547304,2600126,Alcona County,MI,11412.0,Alcona,MI,26001.0,26.0,Alcona,both
1344,MI,alcona,26001,26,2008,1829.234813,2600126,Alcona County,MI,11286.0,Alcona,MI,26001.0,26.0,Alcona,both
1345,MI,alcona,26001,26,2009,2013.542805,2600126,Alcona County,MI,11106.0,Alcona,MI,26001.0,26.0,Alcona,both
1346,MI,alcona,26001,26,2010,1889.294640,2600126,Alcona County,MI,10879.0,Alcona,MI,26001.0,26.0,Alcona,both
1347,MI,alcona,26001,26,2011,2268.387904,2600126,Alcona County,MI,10743.0,Alcona,MI,26001.0,26.0,Alcona,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5009,NC,hyde,37095,37,2009,0.000000,3709537,Hyde County,NC,5774.0,Hyde,NC,37095.0,37.0,Hyde,both
5010,NC,hyde,37095,37,2010,0.000000,3709537,Hyde County,NC,5811.0,Hyde,NC,37095.0,37.0,Hyde,both
5011,NC,hyde,37095,37,2011,0.000000,3709537,Hyde County,NC,5798.0,Hyde,NC,37095.0,37.0,Hyde,both


In [85]:
other_states_shipment_copy = other_states_shipment.copy()

# other_states_shipment_copy["shipment_per_100k"] = (other_states_shipment_copy["QUANTITY"] / other_states_shipment_copy["Population"]) * 100_000
other_states_shipment_copy["shipment_per_100k"] = (
        other_states_shipment_copy["MME"]
    / (other_states_shipment_copy["Population"])
    * 100000
)

other_states_shipment_copy


,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge,shipment_per_100k
index,,,,,,,,,,,,,,,,,
1343,MI,alcona,26001,26,2007,1625.547304,2600126,Alcona County,MI,11412.0,Alcona,MI,26001.0,26.0,Alcona,both,14244.192989
1344,MI,alcona,26001,26,2008,1829.234813,2600126,Alcona County,MI,11286.0,Alcona,MI,26001.0,26.0,Alcona,both,16207.999402
1345,MI,alcona,26001,26,2009,2013.542805,2600126,Alcona County,MI,11106.0,Alcona,MI,26001.0,26.0,Alcona,both,18130.225149
1346,MI,alcona,26001,26,2010,1889.294640,2600126,Alcona County,MI,10879.0,Alcona,MI,26001.0,26.0,Alcona,both,17366.436621
1347,MI,alcona,26001,26,2011,2268.387904,2600126,Alcona County,MI,10743.0,Alcona,MI,26001.0,26.0,Alcona,both,21115.032150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5009,NC,hyde,37095,37,2009,0.000000,3709537,Hyde County,NC,5774.0,Hyde,NC,37095.0,37.0,Hyde,both,0.000000
5010,NC,hyde,37095,37,2010,0.000000,3709537,Hyde County,NC,5811.0,Hyde,NC,37095.0,37.0,Hyde,both,0.000000
5011,NC,hyde,37095,37,2011,0.000000,3709537,Hyde County,NC,5798.0,Hyde,NC,37095.0,37.0,Hyde,both,0.000000


In [86]:
# calculating and displaying results for drug related deaths - year, state and county wise 

other_states_result = other_states_shipment_copy.groupby(["Year", "State", "county_test"])["shipment_per_100k"].sum().reset_index()
other_states_result.head()


,Year,State,county_test,shipment_per_100k
0,2007,MI,Alcona,14244.192989
1,2007,MI,Alger,24229.836384
2,2007,MI,Allegan,17288.908219
3,2007,MI,Alpena,31177.744433
4,2007,MI,Antrim,37241.220111


In [87]:
florida_summary = pd.DataFrame(florida_prescriptions_result.describe()["shipment_per_100k"]).rename(columns={"shipment_per_100k": "Opioid Shipment Per 100K Residents - Florida"})
other_states_summary = pd.DataFrame(other_states_result.describe()["shipment_per_100k"]).rename(columns={"shipment_per_100k": "Opioid Shipment Per 100K Residents - Control States (MI, NC, PA)"})
stats = pd.concat([florida_summary, other_states_summary], axis=1)
stats

,Opioid Shipment Per 100K Residents - Florida,"Opioid Shipment Per 100K Residents - Control States (MI, NC, PA)"
count,469.000000,1750.000000
mean,52582.384891,38532.938813
std,33246.096501,20359.459177
min,0.000000,0.000000
25%,28094.291642,24945.423540
50%,45974.246555,34863.112536
75%,66385.730986,47575.424395
max,227123.239033,172207.387718


In [88]:
# calculating yearwise drug related deaths results for MI
MI_results = other_states_shipment_copy[other_states_shipment_copy["State"] == "MI"].groupby("Year")["shipment_per_100k"].mean()
MI_results

Year
2007    27018.397304
2008    29348.073288
2009    31993.334887
2010    33397.734911
2011    34740.725960
2012    36648.437506
2013    37231.997172
Name: shipment_per_100k, dtype: float64

In [89]:
# calculating yearwise drug related deaths results for NC
NC_results = other_states_shipment_copy[other_states_shipment_copy["State"] == "NC"].groupby("Year")["shipment_per_100k"].mean()
NC_results

Year
2007    33951.760724
2008    36235.322497
2009    41289.639277
2010    44744.885768
2011    47363.290500
2012    50210.871918
2013    52390.596408
Name: shipment_per_100k, dtype: float64

In [90]:
# calculating yearwise drug related deaths results for PA
PA_results = other_states_shipment_copy[other_states_shipment_copy["State"] == "PA"].groupby("Year")["shipment_per_100k"].mean()
PA_results

Year
2007    29971.915624
2008    32792.561514
2009    35599.019479
2010    38655.997910
2011    41463.069356
2012    43663.510165
2013    41922.195033
Name: shipment_per_100k, dtype: float64

In [91]:
fl_result = florida_prescriptions_result.groupby("Year")["shipment_per_100k"].mean().reset_index()
comp_result = other_states_result.groupby(["Year"])["shipment_per_100k"].mean().reset_index()
comp_result
fl_result

,Year,shipment_per_100k
0,2007,38192.192843
1,2008,46442.366133
2,2009,57334.381498
3,2010,73054.089443
4,2011,65767.510504
5,2012,47500.426924
6,2013,39785.726893


In [92]:
florida_summary = pd.DataFrame(fl_result.describe()["shipment_per_100k"]).rename(columns={"shipment_per_100k": "Opioid Shipment per 100k Residents - Florida"})
comp_summary = pd.DataFrame(comp_result.describe()["shipment_per_100k"]).rename(columns={"shipment_per_100k": "Opioid Shipment per 100k Residents - Control States"})
stats = pd.concat([florida_summary, comp_summary], axis=1)
stats

,Opioid Shipment per 100k Residents - Florida,Opioid Shipment per 100k Residents - Control States
count,7.000000,7.000000
mean,52582.384891,38532.938813
std,13227.987417,5363.347735
min,38192.192843,30583.285582
25%,43114.046513,34852.137965
50%,47500.426924,39345.809737
75%,61550.946001,42772.395275
max,73054.089443,44552.409893


In [93]:
# create a scale for number of years before and after 2012 (target year)

# def scale_years(year):
#     if year == 2007:
#         return -3
#     if year == 2008:
#         return -2
#     if year == 2009:
#         return -1
#     if year == 2010:
#         return 0
#     if year == 2011:
#         return 1
#     if year == 2012:
#         return 2
#     if year == 2013:
#         return 3

# fl_result["year relative to policy"] = fl_result["Year"].apply(lambda x: scale_years(x))
# comp_result["year relative to policy"] = comp_result["Year"].apply(lambda x: scale_years(x))

# # double check no nulls in "year relative to policy"

# assert (fl_result["year relative to policy"].isnull().sum() == 0)
# assert (comp_result["year relative to policy"].isnull().sum() == 0)

In [94]:
#fl_result

In [95]:
# # split into before 2011 and after 2011

# wa_b4 = wa_result[wa_result["Year"] <= 2012]
# wa_after = wa_result[wa_result["Year"] >= 2012]


# # wa_after = wa_after[wa_after["Year"] != 2011] # may need to handle this differently
# from sklearn.linear_model import LinearRegression 

# regressor_b4 = LinearRegression() 
# regressor_after = LinearRegression()


# X_b4 = np.array(wa_b4["year relative to policy"]).reshape(-1, 1)
# y_b4 = np.array(wa_b4["shipment_per_100k"]).reshape(-1, 1)

# X_after = np.array(wa_after["year relative to policy"]).reshape(-1, 1)
# y_after = np.array(wa_after["shipment_per_100k"]).reshape(-1, 1)


# regressor_b4.fit(X_b4,y_b4) 
# regressor_after.fit(X_after,y_after) 


# y_pred_b4 = regressor_b4.predict(X_b4)
# y_pred_after = regressor_after.predict(X_after)


In [96]:
# comp_b4 = comp_result[comp_result["Year"] <= 2012]
# comp_after = comp_result[comp_result["Year"] >= 2012]

# # comp_after = comp_after[comp_after["Year"] != 2011] # may need to handle this differently

# regressor_b41 = LinearRegression() 
# regressor_after1 = LinearRegression()


# X_b41 = np.array(comp_b4["year relative to policy"]).reshape(-1, 1)
# y_b41 = np.array(comp_b4["shipment_per_100k"]).reshape(-1, 1)

# X_after1 = np.array(comp_after["year relative to policy"]).reshape(-1, 1)
# y_after1 = np.array(comp_after["shipment_per_100k"]).reshape(-1, 1)


# regressor_b41.fit(X_b41,y_b41) 
# regressor_after1.fit(X_after1,y_after1) 


# y_pred_b41 = regressor_b41.predict(X_b41)
# y_pred_after1 = regressor_after1.predict(X_after)

In [97]:
# y_pred_after1
# X_after1

In [98]:
# plt.xlim(-3, 3)
# #plt.ylim(0, 500)
# plt.title("Opioid Shipment before and after policy implementation in Washington")
# plt.xlabel("year relative to policy")
# plt.ylabel("Opioid Shipment per 100k")
# plt.plot(X_b41, y_pred_b41,color='k', label = "comp before")
# plt.plot(X_after1, y_pred_after1,color='k', label = "comp after")
# plt.plot(X_b4, y_pred_b4,color='b', label = "Washington before")
# plt.plot(X_after, y_pred_after,color='b', label = "Washington after")
# plt.legend()
# plt.show()

# Diff-in-Diff plots

In [99]:
fl_b4 = fl_result[fl_result["Year"] < 2010]
fl_after = fl_result[fl_result["Year"] >= 2010]
comp_b4 = comp_result[comp_result["Year"] < 2010]
comp_after = comp_result[comp_result["Year"] >= 2010]

In [100]:
# Function to plot a vertical line at year of policy implementation
def vertical_line(year):
   
    line = alt.Chart(pd.DataFrame({
    'Year': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Year:Q',
    color=alt.Color('color:N', scale=None)
    )
 
    return line

In [101]:
# creating the vertical line at 2010, the year of implementation of policy changes in Florida
line = vertical_line(2010)


In [102]:
# creating the regression model and calculating the error bands for creating the plots
 
def get_reg_fit_and_ci(data, color, xvar, yvar, legend, alpha=0.05):
   
    # Creating the grid for predicted values
    colour = color
    years = [2007,2008,2009,2010,2011,2012,2013]
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})
 
    # Fitting the model and making the predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)
   
    # Utilizing the predictions to create the points and error bands in the chart
    predictions["Before/After"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar),
            color = alt.Color("Before/After",legend=alt.Legend(title = "States"))
        )
    )
 
    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format=".0f", values=years)),
            y=alt.Y(
                "ci_low",
                title="Opioid Shipment (MME) Per 100K Residents",
                scale=alt.Scale(zero=False),
            ),
            y2="ci_high",
            color=alt.value(f"{color}"),
        )
    )
    chart = ci + reg
    return predictions, chart
 
 


In [103]:
# Using the get_reg_fit_and_ci function to create the charts
 
def build_chart(data, color, xvar, yvar, legend, alpha=0.05):
    fit, reg_chart = get_reg_fit_and_ci(
        data=data, color=color, xvar=xvar, yvar=yvar,  legend=legend, alpha=alpha,
    )
    return reg_chart
 

In [104]:
# creating the final plot for pre-post analysis of drug overdose deaths in Florida
 
florida_before_chart = build_chart(
    fl_b4, "orange", "Year","shipment_per_100k", "Florida",  alpha=0.05
)
 
florida_after_chart = build_chart(
    fl_after, "orange", "Year", "shipment_per_100k", "Florida", alpha=0.05
)

other_states_before_chart = build_chart(
    comp_b4, "blue", "Year","shipment_per_100k", "Control States - MI,NC,PA",  alpha=0.05
)
 
other_states_after_chart = build_chart(
    comp_after, "blue", "Year", "shipment_per_100k", "Control States - MI,NC,PA", alpha=0.05
)
 
florida_final_diff_in_diff_prescriptions = (florida_before_chart + florida_after_chart + other_states_before_chart + other_states_after_chart + line).properties(title="Difference-In-Difference Analysis Of Opioid Shipment In Florida")
 
florida_final_diff_in_diff_prescriptions
 


c:\Users\annap\miniconda3\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [105]:

# x = "Year"
# y = "shipment_per_100k"
# def vertical_line(year):
#     """Function to plot a vertical line at year of policy implementation"""
#     line = alt.Chart(pd.DataFrame({
#     'Date': [year],
#     'color': ["black"]
#     })).mark_rule().encode(
#     x='Date:Q',
#     color=alt.Color('color:N', scale=None)
#     )

#     return line
# def get_reg_fit(data, yvar, xvar, color, title, alpha=0.05):
#     import statsmodels.formula.api as smf

#     # Grid for predicted values
#     x = data.loc[pd.notnull(data[yvar]), xvar]
#     xmin = x.min()
#     xmax = x.max()
#     step = (xmax - xmin) / 100
#     grid = np.arange(xmin, xmax + step, step)
#     predictions = pd.DataFrame({xvar: grid})

#     # Fit model, get predictions
#     model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
#     model_predict = model.get_prediction(predictions[xvar])
#     predictions[yvar] = model_predict.summary_frame()["mean"]
#     predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

#     # Build chart
#     reg = (
#         alt.Chart(predictions)
#         .mark_line()
#         .encode(
#             x=alt.X(xvar, scale=alt.Scale(zero=False), title="Year"),
#             y=alt.Y(
#                 yvar, scale=alt.Scale(zero=False), title="Shipments per 100k Residents"
#             ),
#             color=alt.value(color)
#         ).properties(title=title)
#     )
#     ci = (
#         alt.Chart(predictions)
#         .mark_errorband()
#         .encode(
#             x=xvar,
#             y=alt.Y("ci_low", title=yvar),
#             y2="ci_high",
#             color=alt.value(color),
#         )
#     )
#     chart = ci + reg
#     return predictions, chart
# line = vertical_line(2010)
# fit_fl_b4, reg_chart_fl_b4 = get_reg_fit(
#     fl_b4, yvar="shipment_per_100k", xvar="Year", color="blue", title= "Shipments per 100k Residents - Florida", alpha=0.05
# )


# fit_fl_b4, reg_chart_fl_after = get_reg_fit(
#     fl_after, yvar="shipment_per_100k", xvar="Year", color="blue", title= "Shipments per 100k Residents - Florida",alpha=0.05
# )


In [106]:
# # remaking these plots to change the title for diff in diff
# fit_fl_b4, reg_chart_fl_b4 = get_reg_fit(
#     fl_b4, yvar="shipment_per_100k", xvar="Year", color="blue", title= "Shipments per 100k Residents - Florida (blue) vs. Comp States (green)", alpha=0.05
# )


# fit_fl_b4, reg_chart_wa_after = get_reg_fit(
#     fl_after, yvar="shipment_per_100k", xvar="Year", color="blue", title= "Shipments per 100k Residents - Florida (blue) vs. Comp States (green)",alpha=0.05
# )
# (reg_chart_fl_b4 + reg_chart_wa_after + line).properties(width=700, height=500)

# # texas above
# ################################################################
# # comp below

# fit_comp_b4, reg_chart_comp_b4 = get_reg_fit(
#     comp_b4, yvar="shipment_per_100k", xvar="Year", color="green", title= "Shipments per 100k Residents - Comp States",alpha=0.05
# )
# reg_chart_comp_b4


# fit_comp_after, reg_chart_comp_after = get_reg_fit(
#     comp_after, yvar="shipment_per_100k", xvar="Year", color="green", title= "Shipments per 100k Residents - Comp States",alpha=0.05
# )
# # reg_chart_comp_b4 + reg_chart_comp_after


In [107]:
(reg_chart_fl_b4 + reg_chart_fl_after + reg_chart_comp_b4 + reg_chart_comp_after + line).properties(width=700, height=500)

alt.LayerChart(...)